<h1><center>Recommender Systems YSDA Course</center></h1>
<h1><center>Practice №3 Candidate Generation</center></h1>

<center><img src="https://avatars.mds.yandex.net/get-grocery-goods/2783132/ab847ff6-95e3-4c4e-831a-0576d1949a9e/orig" width="300" /></center>

In this practicte we are going to:
- implement some common recommender system components, known as candidate generators, used for fast early stage retrieval.
- measure some metrics for them
- test them for a user, who really likes fast food!

### Imports and setup

In [ ]:
# if you are running this notebook in colab, datasphere, etc - you need to clone the repository first

# !git clone https://github.com/yandexdataschool/recsys_course
# !pip install -e recsys_course/grocery
# !pip install implicit==0.7.2 transformers==4.49.0 joblib==1.4.2

In [1]:
from abc import ABC, abstractmethod
import random

import numpy as np
import scipy as sp
import polars as pl
import implicit

from tqdm import tqdm

from sklearn.model_selection import train_test_split

### Metrics

One of the main tasks of candidate generators is to find all the potentially relevant items, so that later stages, such as ranking and reranking, could be a lot more complex and require more features and heavier models. By using them we are effectively reducing the number of scored candidates from whole dictionary of items (for TikTok-like systems it's rather impossible) to hundreds or sometimes even thousands.

One of the most popular to evaluate candidate generation is to measure recall or recall@K, and we'll stick by it for this practice and homework. However, evaluating precision or the integral perfomance is also important (see [here](https://t.me/WazowskiRecommends/59), why optimizing recall can be bad).


In [2]:
from grocery.recommender.primitives import Candidate
from grocery.metrics.base import Metric, Evaluator

from grocery.utils.viewer import show_posters, build_item_data

I'll leave the details of the implementation in the imported lib, but the definitions and explanations are below.

```python


class Metric(ABC):
    def __init__(self,
                 k: int | None = None,
                 reduce_function: str = "mean",
                 name: str = "Metric",
                 ):
        # filling all the fields here

    @abstractmethod
    def compute(self,
                predictions: list[Candidate],
                user_id: int | None = None,
                positives: list[int] | None = None
                ) -> float:
        # implement all the computations here, possible to use other data
        # which can be added in the __init__ method


class Evaluator:
    def __init__(self,
                 metrics: list[Metric],
                 ):
        """
        Computes the set of metrics for all the data.
        Args:
            metrics (list[Metric]): metrics to evaluate
        """
    
    def load_test_actions(self, actions: pl.Dataframe):
        """
        Load the data from action-format dataframe. The data is expected to have columns:
        (request_id, user_id, item_id, timestamp) -> per request aggregation will be used.
        Args:
            actions (pl.Dataframe): test dataset, used for metric computation
        """
        

    def evaluate(self, recommend_callable) -> dict[str, float]:
        """
        Runs the evaluation, calling the argument function for each request.
        Function should take user_id and max_k and return a list of recommendations.
        Examples:
            - lambda user_id, n: recommender.recommend(user_id, n)
            - lambda user_id, n: candidate_generator.get_candidates(user_id, n)
        Args:
            recommend_callable (Callable[[int, int], list[Candidate]]): a function to retrieve the ranked items
            with signature like `recommend(user_id, num_items) -> list[candidate]`

        Returns:
            dict[str, float]: dictionary of metric values, aggregated by the metric's reduce function
        """
        

```

</details>

We want to implement the recall as a metric adapter, so we can use it with evaluator with different models later only by implementing the `get_candidates` method.

In [3]:
class Recall(Metric):
    def __init__(self, k: int | None = None):
        # TODO - add your code here
        pass

    def compute(self,
                predictions: list[Candidate],
                positives: list[int],
                user_id: int | None = None,
                ) -> float:
        # TODO - add your code here
        # user_id is not used in this metric
        pass

In [ ]:
# simple test

predictions = [
    Candidate(id=1),
    Candidate(id=2),
    Candidate(id=3),
    Candidate(id=8),
    Candidate(id=9),
    Candidate(id=3),
    Candidate(id=1),
]
positives = {1, 3, 7, 8, 10, 16}
assert np.allclose(float(Recall(k=2).compute(predictions, positives)), 1 / 6)
assert np.allclose(float(Recall(k=4).compute(predictions, positives)), 3 / 6)
assert np.allclose(float(Recall().compute(predictions, positives)), 3 / 6)

### Data

Today's practice, as the last one, will be one the Yandex Lavka dataset of e-grocery recommendations.

**Pros:**
- Items can be recommended multiple times, because users often buy same food, and of the challenges in this type of recsys is to actually find recurring patterns.
- Relatively small, if we are going to focus on only cart update actions - for classic retrieval models it's enough. In ranking seminar we are going to use a full-scale version with views as negatives.

In [5]:
DATA_DIR = "../data/lavka"

# from grocery.utils.dataset import download_and_extract
# download_and_extract(
#     url="https://www.kaggle.com/api/v1/datasets/download/thekabeton/ysda-recsys-2025-lavka-dataset",
#     filename="lavka.zip",
#     dest_dir=DATA_DIR
# )

In [6]:
# read data, leave only the cart updates

data = (
    pl.scan_parquet(f"{DATA_DIR}/train.parquet")
    .filter(pl.col("action_type") == "AT_CartUpdate")
    .collect()
)

item_data = build_item_data(
    pl.scan_parquet("../data/lavka/train.parquet")
)

# leave a subset with only actions and timestamps

mlm_format_data = data.select(
    pl.col("user_id"),
    pl.col("product_id").alias("item_id"),
    pl.col("request_id"),
    pl.lit(1).alias("rating"),
    pl.col("timestamp"),
    pl.col("product_category"),
    pl.col("product_name"),
)

# global timepoint split!

train, test = train_test_split(mlm_format_data.sort("timestamp"), test_size=0.2, shuffle=False)
assert train["timestamp"].max() <= test["timestamp"].min()

We'll also add a fake user, who loves Lipton, Coca-Cola and junk food, plus a helper function to print the results.

In [7]:
sample_user = 1
sample_items = [
    11586455112286391814,
    8817037096041388006,
    11900971701009971240,
    8503917216033445244,
    7316856031090433857,
    8710220968864885647,
    1374883856065269554,
    1810251580582886524,
    4717741123299363694,
    17855866158081617024,
]

mock_user_actions = pl.DataFrame({
    "user_id": [sample_user] * len(sample_items),
    "item_id": sample_items,
    "request_id": np.arange(1, len(sample_items) + 1) * random.randint(63, 76),
    "rating": [1] * len(sample_items),
    "timestamp": [train["timestamp"].max() - 3600] * len(sample_items),
}).select(
    pl.col("user_id").cast(pl.UInt64),
    pl.col("item_id").cast(pl.UInt64),
    pl.col("request_id").cast(pl.UInt64),
    pl.col("rating").cast(pl.Int32),
    pl.col("timestamp"),
)

mock_user_actions = mock_user_actions.join(
    (
        pl.scan_parquet("../data/lavka/train.parquet")
        .select(
            pl.col("product_id").alias("item_id"),
            pl.col("product_category"),
            pl.col("product_name"),
        )
        .unique()
        .collect()
    ),
    on="item_id",
    how="left"
)

train = pl.concat([train, mock_user_actions])

In [ ]:
show_posters(sample_items, item_data)

And this will be our final evaluator with metrics set.

In [8]:
from grocery.metrics.quality import NDCG
from grocery.metrics.aspects import CategoryDiversity, Novelty

evaluator = Evaluator(
    metrics=[
        Recall(10),
        Recall(100),
        NDCG(100),
        Novelty(train, 10),
        Novelty(train, 100),
        CategoryDiversity(train, 10),
        CategoryDiversity(train, 100),
    ]
)

evaluator.load_test_actions(
    test.filter(pl.col("user_id").is_in(train["user_id"].unique()))
)

### Candidate generators 101
Now let's talk business - candidate generators. We want a class with following interface:

```python
class CandidateGenerator:
    @abstractmethod
    def __init__(self):
        """
        Initialize the candidate generator.
        """
        pass

    @abstractmethod
    def extract_candidates(self, object_id: int, n: int = 10) -> list[Candidate]:
        """
        Extract n candidates for the given id (user or item).
        """
        pass
```

During the practice, we'll implement and use two of them:
- `TopPop` - simply returns most popular items, just aggregates user history.
- `KNN` (K nearest neighbors) - extracts the embedding of the object and uses KNN to find objects with high similarity (e.g. dot product).

For second we also need some sort of embeddings for users and items, so we are going to implement two collaborative feedback models and one way to use pretrained language models for content-based generation.

For now, let's deal with the TopPop and measure the metrics - it will be some sort of baseline solution.


In [10]:
from grocery.recommender import CandidateGenerator

In [11]:
class TopPop(CandidateGenerator):
    def __init__(self,
                 interactions: pl.DataFrame,
                 ):
        # TODO - add your code here
        pass

    def extract_candidates(self, object_id: int, n: int = 10) -> list[Candidate]:
        # TODO - add your code here
        pass

In [ ]:
cg = TopPop(train)

metrics = evaluator.evaluate(lambda user_id, n: cg.extract_candidates(user_id, n))
for k, v in metrics.items():
    print(f"{k}: {v:.3f}")

In [ ]:
candidates = cg.extract_candidates(sample_user, 10)
show_posters([c.id for c in candidates], item_data)

#### KNN Candidate Generator
Let's write a lightweight, but actually usable candidate generator! It will be a kNN search, where distance metric is dot-product and closer to 1 is better. This is essentially really useful when you train models to predict score with dot-product of two embedding, just as our ALS model.

A small description:
- Initialization requires `left_embeddings` (user or item ones) and `right_embeddings` (item ones).
- Embeddings are a dictionary with item ids as keys and np.array of fixed dimension as values.
- The same dictionary can be passed as both arguments, if you want to use it for the item-to-item recommendations.
- `extract_candidates` gets the object_id, gets the embedding for it, calculates distances to all other embeddings, sorts them and returns the top n candidates - not really efficient, but it will be the exact closest items.

In [14]:
class DotProductKNN(CandidateGenerator):
    def __init__(self,
                 left_embeddings: dict[int, np.array],
                 right_embeddings: dict[int, np.array],
                 ):
        super().__init__()
        # TODO - add your code here
        pass


    def extract_candidates(
        self,
        object_id: int,
        n: int = 10,
        ) -> list[Candidate]:
        # TODO - add your code here
        pass

Alright, now we actually need some embeddings!

### Content-only candidate generation

Sometimes we may want to find items relevant not for a specific user in general, but rather relevant given the context of the current item the user is looking at - similar videos on YouTube, similar clothing in Wildberries, complements or substitutes for a given product (no microeconomics here, I promise).

What if we do not want to train anything and definitely do not want spend a lot of compute on a regular basis to recalculate the item factors? There is an easy way - you can use content data. Let's try using text from title or a picture of products to find similar products to a certain food item, not using the feedback data. Our first step will be to get a fancy NLP model from somewhere and extract some embeddings. 

In [ ]:
import os
import joblib
from transformers import pipeline


model_id = "DeepPavlov/rubert-base-cased"
pipe = pipeline('feature-extraction', model=model_id)

text_embeddings = {}
if os.path.exists("text_embeddings.pkl"):
    text_embeddings = joblib.load("text_embeddings.pkl")
else:
    products = train.select("product_name", "item_id").unique()
    for text, iid in tqdm(products.iter_rows(), total=len(products)):
        text_embeddings[iid] = np.array(pipe(text))[0, 0]
    joblib.dump(text_embeddings, "text_embeddings.pkl")

But how do we actually get user embeddings for quierying? There are a lot of approaches:
- Gather all the user history, calculate distances from each candidate to every item in history and reduce it (mean / max).
- Calculate some sort of averaged user embedding ahead of time, for example exponentially smoothed mean - the last items have more weight, than the previous ones.

We are going to try out the latter!

In [15]:
user_mean_embeddings = {}

for user_id, items in (
    train
    .select("user_id", "item_id", "timestamp")
    .group_by("user_id")
    .agg(pl.struct("item_id", "timestamp").alias("items"))
).iter_rows():
    items = sorted(items, key=lambda x: x["timestamp"])
    user_embed = np.zeros(768)
    alpha = 0.7
    for item in items:
        user_embed = (1 - alpha) * user_embed + alpha * text_embeddings[item["item_id"]]
    user_mean_embeddings[user_id] = user_embed

In [ ]:
cg = DotProductKNN(user_mean_embeddings, text_embeddings)

metrics = evaluator.evaluate(lambda user_ids, n: cg.batch_extract_candidates(user_ids, n), batch_size=128)
for k, v in metrics.items():
    print(f"{k}: {v:.3f}")

In [ ]:
candidates = cg.extract_candidates(sample_user, 10)
show_posters([c.id for c in candidates], item_data)

Seems to be working and actually is not a bad way to add diversity to your candidates or encourage item exploration, because this candidate is not affected by feedback loop.

### Matrix Factorization

Now to some more robust things, actually used in production - simple and powerful. As you were told in the lecture, matrix factorization is a powerful technique for collaborative filtering.
There are several popular methods, for example:
- Using alternating least squares or ALS.
- Using alternating optimization of log-likelihood instead of MSE.
- Using stochastic optimization of MSE (SVD++ and variations).
- Using Bayesian Personalized Ranking with custom loss function.

Let's start with building a sparse matrix of user and item interactions, and dictionaries to map `id` of object to it's index in the matrix.



In [18]:
def build_matrix_with_mappings(ratings: pl.DataFrame, additive: bool = False):
    users = ratings.select(pl.col("user_id").unique())
    items = ratings.select(pl.col("item_id").unique())
    num_users = len(users)
    num_items = len(items)
    user_id2idx = {row["user_id"]: i for i, row in enumerate(users.iter_rows(named=True))}
    item_id2idx = {row["item_id"]: i for i, row in enumerate(items.iter_rows(named=True))}
    user_idx2id = {v: k for k, v in user_id2idx.items()}
    item_idx2id = {v: k for k, v in item_id2idx.items()}
    R = sp.sparse.lil_array((num_users, num_items))
    for row in ratings.iter_rows(named=True):
        user_idx = user_id2idx[row["user_id"]]
        item_idx = item_id2idx[row["item_id"]]
        if additive:
            R[user_idx, item_idx] += row["rating"]
        else:
            R[user_idx, item_idx] = row["rating"]
    R = R.tocsr()
    return R, (user_id2idx, item_id2idx, user_idx2id, item_idx2id)

### Alternating Least Squares


The idea is that we predict the score with the following formula: $\hat{r}_{ui} = \langle p_u, q_u \rangle + b_{u} + b_{i}$, where $p_u \in \mathbb{R}^d$ и $q_i \in \mathbb{R}^d$ are latent vectors for user $u$ and item $i$ and $b$ are biases.

We will be optimizing MSE between true values and predictions, but with regularization:
$$
L = \sum_{(u, i) \in R} (\hat{r}_{ui} - r_{ui} - b_{u} - b_{i})^2 + \lambda_{emb} \left(\sum_{u \in U} \|p_u\|^2 + \sum_{i \in I} \|q_i\|^2\right) + \lambda_{bias} \left(\sum_{u \in U} b_u^2 + \sum_{i \in I} b_i^2\right)
$$

Assuming you've heard about the model during lecture on candidate generation, we're now going to:
1. Derive the parameter update formulas
2. Code them and implement the model's `fit` method!

#### Step formula for ALS

<details>

<summary> Derivation of the formulas </summary>

User embeddings: 

$$\nabla_{p_u}(L) = \sum_{i \in I} 2q_i(\langle p_u, q_i \rangle + b_u + b_i - r_{ui}) + 2\lambda_{emb} p_u$$

$$\sum_{i \in I} 2(\langle p_u, q_i \rangle + b_u + b_i - r_{ui})q_i + 2\lambda_{emb} p_u = 0$$

$$\sum_{i \in I} p_u q_i^T q_i - \sum_{i \in I} (r_{ui} - b_u - b_i)q_i + \lambda_{emb} p_u = 0$$

$$p_u \left(\sum_{i \in I} q_i^T q_i + \lambda_{emb} I \right) - \sum_{i \in I} (r_{ui} - b_u - b_i) q_i = 0$$

$$p_u = \sum_{i \in I} (r_{ui} - b_u - b_i) q_i \left(\sum_{i \in I} q_i^T q_i + \lambda_{emb} I \right)^{-1}$$

User biases:

$$\nabla_{b_u}(L) = \sum_{i \in I} 2(\langle p_u, q_i \rangle + b_u + b_i - r_{ui}) + 2\lambda_{bias} b_u$$

$$\sum_{i \in I} (\langle p_u, q_i \rangle + b_u + b_i - r_{ui}) + \lambda_{bias} b_u = 0$$

$$\sum_{i \in I} b_u + \lambda_{bias} b_u = \sum_{i \in I} (r_{ui} - \langle p_u, q_i \rangle - b_i) $$

$$ b_u = \frac{1}{ \|I\| + \lambda_{bias} }\sum_{i \in I}(r_{ui} - \langle p_u, q_i \rangle - b_i) $$

The same can be done for item parameters.

</details>

$$ P = (R - B_u - B_i)Q \left(Q^TQ + \lambda_{emb} I \right)^{-1} $$

$$ B_u = \frac{1}{ \|I\| + \lambda_{bias} }(R - PQ^T - B_i) $$

$$ Q = (R - B_u - B_i)^T P \left(P^TP + \lambda_{emb} I \right)^{-1} $$

$$ B_i = \frac{1}{ \|U\| + \lambda_{bias} }(R - PQ^T - B_u) $$

In [19]:
class MatrixFactorization(ABC):
    def __init__(self, dim: int, max_iter: int, lr: float, reg_embeddings: float, reg_biases: float):
        self.lr = lr
        self.dim = dim
        self.max_iter = max_iter
        self.reg_embeddings = reg_embeddings
        self.reg_biases = reg_biases

    def _init_parameters(self, ratings: pl.DataFrame, additive_feedback=False):
        self.R, (
            self.user_id2idx,
            self.item_id2idx,
            self.user_idx2id,
            self.item_idx2id
        ) = build_matrix_with_mappings(ratings, additive=additive_feedback)
        self.n_users, self.n_items = self.R.shape
        self.user_vectors = np.random.normal(size=(self.n_users, self.dim))
        self.item_vectors = np.random.normal(size=(self.n_items, self.dim))
        self.user_biases = np.zeros((self.n_users, 1))
        self.item_biases = np.zeros((self.n_items, 1))

    @abstractmethod
    def fit(self, ratings: pl.DataFrame):
        pass

    def extract_model_to_dicts(self):
        left_biases, left_embeddings = {}, {}
        for user_id, user_idx in self.user_id2idx.items():
            left_embeddings[user_id] = self.user_vectors[user_idx]
            left_biases[user_id] = self.user_biases[user_idx]
        right_biases, right_embeddings = {}, {}
        for item_id, item_idx in self.item_id2idx.items():
            right_embeddings[item_id] = self.item_vectors[item_idx]
            right_biases[item_id] = self.item_biases[item_idx]
        return {
            "left_embeddings": left_embeddings,
            "right_embeddings": right_embeddings,
            "left_biases": left_biases,
            "right_biases": right_biases,
        }

In [20]:
class ALS(MatrixFactorization):
    def __init__(self, 
                 dim: int,
                 max_iter: int,
                 lr: float,
                 reg_embeddings: float,
                 reg_biases: float
                 ):
        super().__init__(dim, max_iter, lr, reg_embeddings, reg_biases)


    def fit(self, ratings: pl.DataFrame):
        # TODO - your code here
        pass

In [ ]:
als = ALS(max_iter=50)
als.fit(train)

embeds = als.extract_model_to_dicts()
cg = DotProductKNN(embeds["left_embeddings"], embeds["right_embeddings"])

In [ ]:
metrics = evaluator.evaluate(lambda user_ids, n: cg.batch_extract_candidates(user_ids, n), batch_size=128)
for k, v in metrics.items():
    print(f"{k}: {v:.3f}")

In [ ]:
candidates = cg.extract_candidates(sample_user, 10)
show_posters([c.id for c in candidates], item_data)

There is an interesting property with iALS models: the norm of vectors correlates with item popularity or user popularity.

In [ ]:
item_popularity = train.group_by("item_id").agg(pl.col("rating").count().alias("popularity"))
item_popularity = item_popularity.with_columns(
    norms=np.array([np.linalg.norm(embeds["right_embeddings"][iid]) for iid in item_popularity["item_id"]])
)

print(item_popularity["popularity", "norms"].corr())

user_hotnesss = train.group_by("user_id").agg(pl.col("rating").count().alias("hotness"))
user_hotnesss = user_hotnesss.with_columns(
    norms = np.array([np.linalg.norm(embeds["left_embeddings"][uid]) for uid in user_hotnesss["user_id"]])
)

print(user_hotnesss["hotness", "norms"].corr())

### Logistic Matrix Factorization

We model the probability of the interaction as follows:

$$P(r_{ui} = 1 | p_u, q_i, b_u, b_i) = \sigma \left( \langle p_u, q_u \rangle + b_{u} + b_{i} \right)$$

Where $\sigma(x)$ is a sigmoid function, and other parameters are the same as in ALS.

$$\sigma(x) = \frac{1}{1 + \exp(-x)}$$

<details>

<summary> Derivation of the formulas </summary>

Let also $x_{ui}$ be a logit of the model:

$$x_{ui} = \langle p_u, q_u \rangle + b_{u} + b_{i}$$
$$\nabla_{p_{u}}(x_{ui}) = q_i$$
$$\nabla_{q_{i}}(x_{ui}) = p_u$$
$$\nabla_{b_{u}}(x_{ui}) = 1$$
$$\nabla_{b_{i}}(x_{ui}) = 1$$

For a binary observation, likelihood is modeled like this:

$$p(r_{ui} | p_u, q_i, b_u, b_i) =  \sigma ( x_{ui} )^{r_{ui}} \left( 1 - \sigma (x_{ui}) \right)^{1 - r_{ui}}$$

$$ l_{ui} = r_{ui} \log \left( \sigma (x_{ui})\right) + (1 - r_{ui}) \log \left( 1 - \sigma (x_{ui}) \right)$$

The loss will be the log-likelihood with subtracted regularization:
$$
L = \sum_{(u, i) \in R} \left[ r_{ui} \log \left( \sigma (x_{ui})\right) + (1 - r_{ui}) \log \left( 1 - \sigma (x_{ui}) \right) \right] - \lambda_{emb} \left(\sum_{u \in U} \|p_u\|^2 + \sum_{i \in I} \|q_i\|^2\right) - \lambda_{bias} \left(\sum_{u \in U} b_u^2 + \sum_{i \in I} b_i^2\right)
$$

It's possible to solve this alternating the gradient **ascent** (or descent, if negative log-likelihood is used with added regularization terms) steps between users and items, freezing the other parameters when making a step. As $\sigma$ is a logistic function, we can easily derive the derivative by the argument:

$$\nabla_{x_{ui}}(l_{ui}) = r_{ui} - \sigma (x_{ui})$$
$$\nabla_{p_{u}}(L) = \sum_{i \in I} \left(r_{ui} - \sigma (x_{ui}) \right) q_i - \lambda_{emb} p_u $$
$$\nabla_{b_{u}}(L) = \sum_{i \in I} \left(r_{ui} - \sigma (x_{ui}) \right) - \lambda_{bias} b_u $$

The derivation of item gradients is the same.

</details>

$$p_{u} := p_{u} + \eta \left[ \sum_{i \in I} \left(r_{ui} - \sigma (x_{ui}) \right) q_i - \lambda_{emb} p_u \right]$$
$$b_{u} := b_{u} + \eta \left[ \sum_{i \in I} \left(r_{ui} - \sigma (x_{ui}) \right) - \lambda_{bias} b_u \right]$$
$$q_{i} := q_{i} + \eta \left[ \sum_{i \in I} \left(r_{ui} - \sigma (x_{ui}) \right) p_u - \lambda_{emb} q_i \right]$$
$$b_{i} := b_{i} + \eta \left[ \sum_{i \in I} \left(r_{ui} - \sigma (x_{ui}) \right) - \lambda_{bias} b_i \right]$$

Next - implementation:

In [25]:
class LogisticMF(MatrixFactorization):
    def __init__(self,
                 dim: int = 128,
                 max_iter: int = 300,
                 lr: float = 0.003,
                 reg_embeddings: float = 0.01,
                 reg_biases: float = 0.01,
                 ):
        # TODO - your code here
        pass

    def fit(self, ratings: pl.DataFrame, additive_feedback=False):
        # TODO - your code here
        pass


In [ ]:
lmf = LogisticMF(max_iter=50)
lmf.fit(train)

embeds = lmf.extract_model_to_dicts()
cg = DotProductKNN(embeds["left_embeddings"], embeds["right_embeddings"])

In [ ]:
metrics = evaluator.evaluate(lambda user_ids, n: cg.batch_extract_candidates(user_ids, n), batch_size=128)
for k, v in metrics.items():
    print(f"{k}: {v:.3f}")

In [ ]:
candidates = cg.extract_candidates(sample_user, 10)
show_posters([c.id for c in candidates], item_data)

In homework, your assingment will be to implement the other mentioned algorithm - BPR. You will also implement two other somewhat similar approaches - EASE and SLIM, which build an item-to-item similarity matrix and find items, similar to user history.

Save this notebook, cause you'll need the metrics and the models for comparison!

### Comparing with some benchmarks
Let's compare the quality of our solution with the `implicit` library - a popular way to do matrix factorization.

In terms of speed, GPU support and scaling `implicit` is 100% better, but we should not fall back on quality too much.

In [ ]:
additive = True

R, (user_id2idx, item_id2idx, user_idx2id, item_idx2id) = build_matrix_with_mappings(train, additive=additive)
model = implicit.als.AlternatingLeastSquares(factors=128, iterations=100)
model.fit(R)

left_embeddings = {}
for user_id, user_idx in user_id2idx.items():
    left_embeddings[user_id] = model.user_factors[user_idx]
right_embeddings = {}
for item_id, item_idx in item_id2idx.items():
    right_embeddings[item_id] = model.item_factors[item_idx]

cg = DotProductKNN(left_embeddings, right_embeddings)
metrics = evaluator.evaluate(lambda user_ids, n: cg.batch_extract_candidates(user_ids, n), batch_size=128)
for k, v in metrics.items():
    print(f"{k}: {v:.3f}")

In [ ]:
candidates = cg.extract_candidates(sample_user, 10)
show_posters([c.id for c in candidates], item_data)